<h4 class='prehead'>SM286D &middot; Introduction to Applied Mathematics with Python &middot; Spring 2020 &middot; Foraker/Traves/Uhan</h4>

<h3 class='lesson'>Project 3.</h3>

<h1 class='lesson_title'>Linear Programming</h1>

__Mathematical goals.__ Linear programming.

__Programming goals.__ Create and solve optimization problems with Pyomo.

---

## Motivation

__[Pyomo](http://www.pyomo.org/)__ is a Python-based, open-source optimization modeling language that was developed at the Sandia National Laboratories.  Pyomo is used by students at many institutions, including Rose-Hulman Institute of Technology, University of California Davis, University of Texas, Iowa State University, and the Naval Postgraduate School. It is also used by various government agencies and national laboratories. One of the nicest features of Pyomo is the ability to use just about any commercial solver (including Gurobi and CPLEX) or open-source solver (including GLPK and COIN-OR).

You will be using Pyomo to formulate and solve models in SA305 and SA405 as part of the operations research major curriculum at the Naval Academy.

For this project, you will walk through the basics of optimization modeling with Pyomo, and then write your own Pyomo model for a linear program.

---

## Introduction to optimization modeling with Pyomo

To illustrate the basics of optimization modeling with Pyomo, we will model Example 1.1 from Section 1.2 of *Deterministic Operations Research - Models and Methods in Linear Optimization* by David J. Rader Jr.

__Example 1.1.__ Farmer Jones decides to supplement his income by baking and selling two types of cakes, chocolate and vanilla.  Each chocolate cake sold gives a profit of \\$3, and the profit on each vanilla cake sold is \\$5.  Each chocolate cake requires 20 minutes of baking time and uses 4 eggs and 4 pounds of flour, while each vanilla cake requires 40 minutes of baking time and uses 2 eggs and 5 pounds of flour.  If Farmer Jones has available only 260 minutes of baking time, 32 eggs, and 40 pounds of flour, how many of each type of cake should be baked in order to maximize Farmer Jones' profit?

Decision variables:
\begin{align*}
C & = \text{number of chocolate cakes to bake}\\
V & = \text{number of vanilla cakes to bake}
\end{align*}

Objective function and constraints:
\begin{alignat*}{2}
\text{maximize} \quad & 3 C + 5 V &\qquad& \text{(maximize profit)}\\
\text{subject to} \quad & 4 C + 2 V \le 32 &\qquad& \text{(eggs)}\\
& 4 C + 5 V \le 40 &\qquad& \text{(flour)}\\
& 20 C + 40 V \le 260 &\qquad& \text{(baking time)}\\
& C, V \ge 0 &\qquad& \text{(nonnegativity)}
\end{alignat*}

In order to model this problem, you should already have Pyomo and GLPK installed on your machine.  If you do not have these packages installed, please refer to Lesson 0.

We begin by importing the Pyomo modeling environment.

In [ ]:
import pyomo.environ as pyo

Next, we initialize a __concrete model__.  Concrete models are immediately constructed, so data must be present _at the time_ the model components are defined. The variable `model` stores information about the model we will construct. The variable doesn't have to be named `model`, but we will typically use `model` by convention.

In [ ]:
model = pyo.ConcreteModel()

You can verify what type of model you have using the code below.

In [ ]:
type(model)

Next, we define the **decision variables** for our model.

In [ ]:
model.C = pyo.Var(within=pyo.NonNegativeReals)
model.V = pyo.Var(within=pyo.NonNegativeReals)

The word `within` is optional and sets the variable domain.  You can think of `within` as an alias for domain.  There are several pre-defined domains besides `NonNegativeReals`.  For example, `Binary` is another domain you will frequently encounter.

Next, we will define the **objective function**.

In [ ]:
def obj_rule(model):
    return 3 * model.C + 5 * model.V
model.obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)

Note that the objective function is a Python function.  If `sense` is omitted, Pyomo assumes that it is a minimization problem.

Next, we define the **constraints** for the model.  First, we write the constraint for the eggs.

In [ ]:
def egg_limit_rule(model):
    return 4 * model.C + 2 * model.V <= 32
model.egg_limit = pyo.Constraint(rule=egg_limit_rule)

Next, we write the constraint for the flour.

In [ ]:
def flour_limit_rule(model):
    return 4 * model.C + 5 * model.V <= 40
model.flour_limit = pyo.Constraint(rule=flour_limit_rule)

Finally, we write the constraint for the baking time.

In [ ]:
def baking_time_limit_rule(model):
    return 20 * model.C + 40 * model.V <= 260
model.baking_time_limit = pyo.Constraint(rule=baking_time_limit_rule)

Note that each constraint is a Python function.

Now we need to specify a solver.  There are several solvers supported by Pyomo, but we will use GLPK for this example.

In [ ]:
opt = pyo.SolverFactory("glpk")

Next, we call the solver.

In [ ]:
results = opt.solve(model, tee=True)

Then we want to see the results of the optimization.

In [ ]:
model.pprint()

Using `model.pprint` can produce a significant amount of output, not all of which is particularly useful.  It is usually better to look at the decision variables directly by using `.pprint()`.

In [ ]:
model.C.pprint()

In [ ]:
model.V.pprint()

You can also get the value of the objective function directly using the `.expr()` method as shown below.

In [ ]:
model.obj.expr()

So the optimal solution for Farmer Jones is to make 5 chocolate and 4 vanilla cakes, generating a profit of \\$35.  Another way to view the results of the optimization is to use `print()`.  The code below is an example of how to generate this type of output.

In [ ]:
print(f'Farmer Jones should make {pyo.value(model.C):.2f} chocolate and '
      f'{pyo.value(model.V):.2f} vanilla cakes, generating a profit of {model.obj():.2f}.')

---

## Your assignment

Use the knowledge gained by reading through the example above to solve Example 2.1 from Section 2.1 of *Deterministic Operations Research - Models and Methods in Linear Optimization* by David J. Rader Jr.  

__Example 2.1.__ The Terre Haute Door Company (THDC) designs three types of steel doors: Standard, High Security, and Maximum Security.  Each door requires different amounts of machine and labor time and has different profit margins; this information is given in the following table.

\begin{array}{l|c|c|c|c|c}
& \mbox{Machine 1 Hours} & \mbox{Machine 1 Manpower} & \mbox{Machine 2 Hours} & \mbox{Machine 2 Manpower} & \mbox{Profit Margin} \\
\hline
\mbox{Standard} & 3.5 & 5 & 4 & 6 & $35 \\
\hline
\mbox{High Security} & 6 & 8 & 5 & 7 & $45 \\
\hline
\mbox{Maximum Security} & 8 & 11 & 6 & 9 & $65 \\
\hline
\end{array}

Each door must go through both machine 1 and machine 2 before it can be sold.  Each worker is assigned to work on only one of the doors, which means they work on both machines.  In addition, management has decided not to sell more Maximum Security doors than the combined total of Standard and High Security doors sold, in order to keep demand high for Standard and High Security doors.  THDC has available to it only 120 hours per week on machine 1 and 100 hours on machine 2 before required maintenance, and 280 hours of manpower available per week.  If we assume that we can sell every door that we make, how many of each door should be produced each week in order to maximize profits?

We can model this problem as the following linear program.

Decision variables:
\begin{align*}
x_1 & = \rm{number~of~Standard~doors~produced}\\
x_2 & = \rm{number~of~High~Security~doors~produced}\\
x_3 & = \rm{number~of~Maximum~Security~doors~produced}
\end{align*}

Objective function and constraints are:
\begin{alignat*}{2}
\text{maximize} \quad & 35 x_1 + 45 x_2 + 65 x_3 &\qquad& \text{(objective)} \\
\text{subject to} \quad & 3.5 x_1 + 6 x_2 + 8 x_3 \leq 120 &\qquad& \text{(machine 1)} \\
 & 4 x_1 + 5 x_2 + 6 x_3 \leq 100 &\qquad& \text{(machine 2)} \\
 & 11 x_1 + 15 x_2 + 20 x_3 \leq 280 &\qquad& \text{(labor)} \\
 & x_3 \leq x_1 + x_2 &\qquad& \text{(management)} \\
 & x_1, x_2, x_3 \ge 0 &\qquad& \text{(nonnegativity)}
\end{alignat*}

Formulate and solve this model by completing the tasks given below.

__(1)__  Begin by importing the Pyomo modeling environment. (2 points)

__(2)__ Define the variable `model` as a concrete model.  (2 points)

__(3)__ Define the decision variables, $x_1$, $x_2$, and $x_3$.  Make sure you properly specify their domains. (3 points)

__(4)__ Define the objective function.  Make sure you specify the correct `sense`. (3 points)

__(5)__ Define the `machine 1` constraint. (3 points)

__(6)__  Define the `machine 2` constraint. (3 points)

__(7)__ Define the `labor` constraint. (3 points)

__(8)__ Define the `management` constraint. (3 points)

__(9)__ Specify that you will use the `glpk` solver for this model. (2 points)

__(10)__ Generate the results of the optimization by calling the solver. (2 points)

__(11)__ Use a nicely formatted print statement to display the results of the optimization.  Your output must include how many of each type of door should be produced as well as the profit generated by those sales. You __must__ use 2 decimal places for each of the numerical outputs, and you __must not__ hard-code your print statement in order to receive full credit. (4 points)

For example, your output could say something like:

```
THDC should produce 2.02 Standard, 5.06 High Security, and 6.08 Maximum Security Doors.  This will generate a profit of $867.23.
```

---

## When you're finished

- Make sure your notebook runs from top to bottom with no errors. One way to accomplish this is to click on __Kernel &#8594; Restart & Run All__. This will restart Python, and run your notebook from top to bottom.

- Please acknowledge any collaborators below:

<span style="color:red"><strong>LIST YOUR COLLABORATORS IN THIS CELL. DOUBLE-CLICK TO EDIT.</strong></span>

- When you're ready, submit this notebook using the SM286D Submission Form linked on the [class website](https://www.usna.edu/users/math/uhan/sm286d/).

---

## Grading

This assignment is worth a total of 30 points. The point values for items __(1)__-__(11)__ are given above.